<br>

# Maps

In [ ]:
!pip3 install open-geodata --upgrade
!pip3 install sp-piracicaba

In [ ]:
import os
import json
import fiona
import folium
import pandas as pd
import branca as bc
import seaborn as sns
import geopandas as gpd
from datetime import date
from folium import plugins

In [ ]:
from open_geodata import geo, lyr

In [ ]:
from paths import *

In [ ]:
import src.sp_piracicaba.lyr

In [ ]:
list_shp = geo.get_dataset_names_others('sp_piracicaba')
list_shp

In [ ]:
# List geodataframes
geo.get_dataset_names()
gdf = geo.load_dataset_others('sp_piracicaba', 'divisa_municipal.7z')

<br>

# Layers

<br>

# Mapa Plano Diretor

In [ ]:
def get_map(input_shp):
    # Input
    gdf = gpd.read_file(input_shp)
    gdf = gdf.to_crs(epsg=4326)
    sw = gdf.bounds[['miny', 'minx']].min().values.tolist()
    ne = gdf.bounds[['maxy', 'maxx']].max().values.tolist()
    bounds = [sw, ne]
    
    # Zoom
    min_zoom = 10
    max_zoom = 18

    # Create Map
    m = folium.Map(
        #location=list_centroid,
        #zoom_start=10,
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        max_bounds=True,
        #zoom_delta=0.1,
        min_lat=bounds[0][0]*(101/100),
        min_lon=bounds[0][1]*(101/100),
        max_lat=bounds[1][0]*(99/100),
        max_lon=bounds[1][1]*(99/100),
        tiles=None,
    )
    
    # Add Base Layers
    m.add_child(lyr.base.google_hybrid(min_zoom, max_zoom))
    m.add_child(lyr.base.google_terrain(min_zoom, max_zoom))
    m.add_child(lyr.base.google_streets(min_zoom, max_zoom))
    m.add_child(lyr.base.google_satellite(min_zoom, max_zoom))
    
    # Plano Diretor
    m.add_child(src.sp_piracicaba.lyr.divisa_municipal())
    m.add_child(src.sp_piracicaba.lyr.macrozona())
    m.add_child(src.sp_piracicaba.lyr.divisa_abairramento())
    m.add_child(src.sp_piracicaba.lyr.divisa_urbano_rural())
    m.add_child(src.sp_piracicaba.lyr.perimetro_urbano())
    
    # Plugins
    m.fit_bounds(bounds)
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
    ).add_to(m)
    folium.LayerControl(
        position='topright',
        collapsed=False
    ).add_to(m)
    return m

In [ ]:
m = get_map(os.path.join(output_path_shp, 'Divisa_Municipal.shp'))

# Results
m.save(os.path.join(output_path_map, 'map_pd_piracicaba.html'))
m